In [1]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from keras import backend as K
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier

#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

/home/titli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [3]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [4]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [5]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [6]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples].reshape(15998,1000,1)
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:].reshape(3999,1000,1)
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997,)


In [7]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


Training Model

In [8]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[10000:19000], ham_index[9997:19000]), axis=0)
keras_data_test_set = keras_data.iloc[new_index]
labels_test_set = keras_data_test_set.isspam
keras_data_test_set.head()

,Subject,isspam
19223,RE: Daily Summary of Risk Data,0
19224,Softwares CDS all software under 15$ and 99$!,1
19226,Hotel Room Bargains at up to 70% off! Save in...,1
19227,RE: Greeley Gas Company,0
19228,"Any Software just in under 15-99$, Xp-adobe etc",1


In [9]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_test_set.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_test_set = to_categorical(np.asarray(labels_test_set))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels_test_set = labels_test_set[indices]
x_test = data.reshape(17088,1000,1)

Found 15425 unique tokens.
Shape of data tensor: (17088, 1000)
Shape of label tensor: (19997,)


In [10]:
def no_embedding(fl1=16, fl2=16, fl3=16, dl=16, optimizer= 'RMSprop', kl = 5, layer =1 ):
    inp =  Input(shape=(1000, 1))
    if (layer == 1):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
    elif (layer == 2):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
    else:
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl3, kernel_size = kl, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(units = dl, activation='relu')(x)
    preds = Dense(1, activation='tanh')(x)
    model = Model(inp, preds)
    model.compile(loss= 'binary_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
   
    return model

In [11]:
def kera_trainable_params(model):
    trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])) 
    return trainable_count

In [12]:
def F1_score(pred_test_y, actuals):

    predictions =[]
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0

    for i in range(len(pred_test_y)):
        if (pred_test_y[i]>=0.5):
            predictions.append(1)
        else:
            predictions.append(0)
    
    for i in range (len(pred_test_y)):
        if ((predictions[i]==1) & (actuals[i]==1)):
            true_pos = true_pos+1
        elif((predictions[i]==0) & (actuals[i]==0)):
            true_neg = true_neg+1
        elif((predictions[i]==1) & (actuals[i]==0)):
            false_pos = false_pos +1
        elif((predictions[i]==0) & (actuals[i]==1)):
            false_neg = false_neg+1
    try:
        prec=true_pos/(true_pos+false_pos)
        recall = true_pos/(true_pos+false_neg)
        accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
        F1=2*(prec*recall/(prec+recall))
        FPR = false_pos/(false_pos+true_neg)
    except ZeroDivisionError:
        prec=0
        recall = 0
        accur=0
        F1=0
    return (true_pos, false_pos, true_neg, false_neg, prec)

In [13]:
def loss_edit(out,model_test_y, labels_test_set):
    loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' in s]
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
    for l in loss_list:
        train_loss .append(out.history[l])
    for l in val_loss_list:
        val_loss .append(out.history[l])
    for l in acc_list:
        train_acc.append(out.history[l])
    for l in val_acc_list:
        val_acc .append(out.history[l])
    true_pos, false_pos, true_neg,false_neg,prec  = F1_score(model_test_y, labels_test_set)
    line  = { 'train_loss': reduce(operator.concat, train_loss)[-1],'val_loss': reduce(operator.concat, val_loss)[-1], \
             'train_acc': reduce(operator.concat, train_acc)[-1], 'val_acc': reduce(operator.concat, val_acc)[-1],\
            'true_pos':true_pos, 'false_pos':false_pos, 'true_neg':true_neg,\
             'false_neg':false_neg, 'prec':prec}
    return (line)

In [ ]:
# bounds for hyper-parameters in mnist model
# the bounds dict should be in order of continuous type and then discrete type
# create model
# define the grid search parameters
layers = [1,2,3]
fl1s = [16,32,64, 128]
fl2s = [16,32,64, 128]
fl3s = [16,32,64,128]
kls = [5]
dls = [16,32,64,128]
#sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
optimizers = ['Nadam', 'Adam', 'RMSprop', 'SGD'] #, 'Adadelta', 'Adam', 'Adamax', 'Nadam']
loss_data = []
params = []
trainable_count = []
for fl1, fl2, fl3, kl, dl, optimizer,layer in product(fl1s,fl2s,fl3s,kls,dls,optimizers,layers):
    kwargs = dict(fl1=fl1, fl2= fl2, fl3=fl3, kl=kl, dl=dl, optimizer= ''.join(optimizer), layer=layer)
    params.append(kwargs)
    model = no_embedding(**kwargs)
    trainable_count.append(kera_trainable_params(model))
    history = model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
    model_test_y = model.predict(x_test, batch_size=16, verbose=1)
    loss_data.append(loss_edit(history,model_test_y,labels_test_set))
    

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 10s 640us/step - loss: 7.7547 - acc: 0.0731 - val_loss: 7.6546 - val_acc: 0.0550
Epoch 2/2
17088/17088 [==============================] - 2s 133us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 10s 629us/step - loss: 7.6607 - acc: 0.0633 - val_loss: 7.5004 - val_acc: 0.0663
Epoch 2/2
17088/17088 [==============================] - 3s 177us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 11s 685us/step - loss: 8.2604 - acc: 0.0614 - val_loss: 8.1820 - val_acc: 0.0585
Epoch 2/2
17088/17088 [==============================] - 3s 193us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 10s 599us/step - loss: 8.2790 - acc: 0.0666 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [================

Epoch 2/2
17088/17088 [==============================] - 4s 250us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 13s 816us/step - loss: 6.9941 - acc: 0.1148 - val_loss: 6.8283 - val_acc: 0.1155
Epoch 2/2
17088/17088 [==============================] - 5s 293us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 14s 855us/step - loss: 3.8470 - acc: 0.3450 - val_loss: 0.9613 - val_acc: 0.4991
Epoch 2/2
17088/17088 [==============================] - 5s 300us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 12s 766us/step - loss: 8.2659 - acc: 0.0519 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 4s 258us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 13s 817us/step - loss: 8.2303 - acc: 0.0608 - val

17088/17088 [==============================] - 7s 385us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 17s 1ms/step - loss: 8.2688 - acc: 0.0522 - val_loss: 8.1820 - val_acc: 0.0495
Epoch 2/2
17088/17088 [==============================] - 7s 416us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 18s 1ms/step - loss: 5.9506 - acc: 0.1731 - val_loss: 1.6541 - val_acc: 0.4456
Epoch 2/2
17088/17088 [==============================] - 8s 471us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 17s 1ms/step - loss: 7.7602 - acc: 0.0542 - val_loss: 7.6608 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 7s 400us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 17s 1ms/step - loss: 7.5303 - acc: 0.0680 - val_loss: 7.1043 - va

15998/15998 [==============================] - 19s 1ms/step - loss: 8.2686 - acc: 0.0586 - val_loss: 8.1820 - val_acc: 0.0568
Epoch 2/2
17088/17088 [==============================] - 9s 555us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 21s 1ms/step - loss: 8.2686 - acc: 0.0649 - val_loss: 8.1820 - val_acc: 0.0645
Epoch 2/2
17088/17088 [==============================] - 9s 531us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 19s 1ms/step - loss: 8.2708 - acc: 0.0558 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 9s 520us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 20s 1ms/step - loss: 7.6864 - acc: 0.0656 - val_loss: 7.4721 - val_acc: 0.0688
Epoch 2/2
17088/17088 [==============================] - 9s 507us/step
Train on 15998 samples, validate on

Epoch 2/2
17088/17088 [==============================] - 11s 636us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 24s 2ms/step - loss: 8.2677 - acc: 0.0588 - val_loss: 8.1820 - val_acc: 0.0570
Epoch 2/2
17088/17088 [==============================] - 10s 605us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 20s 1ms/step - loss: 8.1892 - acc: 0.0673 - val_loss: 8.1820 - val_acc: 0.0380
Epoch 2/2
17088/17088 [==============================] - 9s 549us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 21s 1ms/step - loss: 8.2653 - acc: 0.0520 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 10s 582us/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 23s 1ms/step - loss: 8.2654 - acc: 0.0589 - val_loss

...

15998/15998 [==============================] - 109s 7ms/step - loss: 7.6208 - acc: 0.0658 - val_loss: 7.4924 - val_acc: 0.0618
Epoch 2/2
17088/17088 [==============================] - 53s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 100s 6ms/step - loss: 7.3721 - acc: 0.0949 - val_loss: 8.1820 - val_acc: 0.0825
Epoch 2/2
17088/17088 [==============================] - 55s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 91s 6ms/step - loss: 7.7395 - acc: 0.0628 - val_loss: 7.6990 - val_acc: 0.0455
Epoch 2/2
17088/17088 [==============================] - 51s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 98s 6ms/step - loss: 8.2624 - acc: 0.0624 - val_loss: 8.1820 - val_acc: 0.0598
Epoch 2/2
17088/17088 [==============================] - 54s 3ms/step
Train on 15998 samples, validate on 3

17088/17088 [==============================] - 56s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 106s 7ms/step - loss: 6.8299 - acc: 0.2113 - val_loss: 7.6589 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 59s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 97s 6ms/step - loss: 8.2366 - acc: 0.0596 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 55s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 106s 7ms/step - loss: 8.2662 - acc: 0.0524 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 57s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 108s 7ms/step - loss: 8.2572 - acc: 0.0655 - val_loss: 8.1820 - val

17088/17088 [==============================] - 63s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 104s 6ms/step - loss: 8.2511 - acc: 0.0539 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 59s 3ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 114s 7ms/step - loss: 8.2637 - acc: 0.0536 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 63s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 123s 8ms/step - loss: 7.2190 - acc: 0.0978 - val_loss: 6.5431 - val_acc: 0.2118
Epoch 2/2
17088/17088 [==============================] - 62s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 105s 7ms/step - loss: 8.2686 - acc: 0.0525 - val_loss: 8.1820 - va

15998/15998 [==============================] - 115s 7ms/step - loss: 8.2614 - acc: 0.0528 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 63s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 116s 7ms/step - loss: 8.2628 - acc: 0.0594 - val_loss: 8.1820 - val_acc: 0.0568
Epoch 2/2
17088/17088 [==============================] - 64s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 116s 7ms/step - loss: 6.8211 - acc: 0.1116 - val_loss: 8.1820 - val_acc: 0.0000e+00
Epoch 2/2
17088/17088 [==============================] - 65s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 117s 7ms/step - loss: 7.7444 - acc: 0.0614 - val_loss: 7.6607 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 64s 4ms/step
Train on 15998 samples, validat

17088/17088 [==============================] - 68s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 127s 8ms/step - loss: 8.2171 - acc: 0.0614 - val_loss: 8.1779 - val_acc: 0.0498
Epoch 2/2
17088/17088 [==============================] - 69s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 126s 8ms/step - loss: 6.8892 - acc: 0.1350 - val_loss: 5.9470 - val_acc: 0.1865
Epoch 2/2
17088/17088 [==============================] - 70s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 126s 8ms/step - loss: 8.2686 - acc: 0.0517 - val_loss: 8.1820 - val_acc: 0.0493
Epoch 2/2
17088/17088 [==============================] - 69s 4ms/step
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 125s 8ms/step - loss: 7.8326 - acc: 0.0559 - val_loss: 7.6582 - va

In [ ]:
params_pd = pd.DataFrame(params)
loss_pd= pd.DataFrame(loss_data)
trainable_pd = pd.DataFrame({'trainable_params': trainable_count})

In [ ]:
no_embeddings = pd.concat([params_pd, loss_pd, trainable_pd], axis=1, join='inner')
no_embeddings.to_csv('no_embeddings_models.csv', index=False)


In [13]:
no_embeddings = pd.read_csv('no_embeddings_models.csv')
index = (no_embeddings['train_acc']>no_embeddings['val_acc'])
new1 = no_embeddings[index]
index1 = (new['train_loss']<new['val_loss'])
new1 = new[index1]
new1.sort_values(['val_loss'])
#no_embeddings_sort = no_embeddings.sort_values(['val_loss'])
#no_embeddings_sort

,dl,fl1,fl2,fl3,kl,layer,optimizer,false_neg,false_pos,prec,train_acc,train_loss,true_neg,true_pos,val_acc,val_loss,trainable_params
617,128,16,128,16,5,3,Adam,8981,39,0.530120,0.506688,0.694596,8024,44,0.496374,0.704552,23025
806,128,32,16,16,5,3,Nadam,8965,64,0.483871,0.483873,1.003907,7999,60,0.466867,1.319187,6369
17,32,16,16,16,5,3,Adam,6580,2171,0.529679,0.211526,6.126347,5892,2445,0.142786,6.318781,3265
437,16,16,64,32,5,3,Adam,6713,2067,0.527974,0.166208,6.398598,5996,2312,0.120780,6.519629,16097
248,16,16,32,32,5,3,RMSprop,8502,464,0.529889,0.162083,6.342405,7599,523,0.125281,6.631214,8385
134,128,16,16,64,5,3,Nadam,7318,1473,0.536792,0.418490,2.417076,6590,1707,0.090023,6.748346,15025
647,32,16,128,32,5,3,SGD,9025,0,0.000000,0.202463,6.382880,8063,0,0.164791,6.779000,32065
19,32,16,16,16,5,2,RMSprop,8552,442,0.516940,0.132517,6.748979,7621,473,0.109777,6.897287,1969
787,32,32,16,16,5,2,RMSprop,8617,365,0.527814,0.132267,6.790238,7698,408,0.107527,6.927578,3345
187,128,16,16,128,5,2,RMSprop,8728,280,0.514731,0.103700,7.064635,7783,297,0.086272,7.095588,3697
